# Slack
You need to fill in the [form](https://docs.google.com/forms/d/1OmT8ODmVBNgl0eOmZT51JMTHUSA_eNrHTcDRnmNDMgQ) to get invitated

Slack url: https://rt-portal.slack.com/

# Pandas Cheet Sheat
https://github.com/pandas-dev/pandas/raw/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

## Essential functionality

In this section, I’ll walk you through the fundamental mechanics of interacting with
the data contained in a Series or DataFrame. Upcoming chapters will delve more deeply
into data analysis and manipulation topics using pandas. This book is not intended to
serve as exhaustive documentation for the pandas library; I instead focus on the most
important features, leaving the less common (that is, more esoteric) things for you to
explore on your own.

### Reindexing

A critical method on pandas objects is **reindex** , which means to create a new object
with the data *conformed* to a new index. Consider a simple example from above:

In [ ]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

Calling **reindex** on this Series rearranges the data according to the new index, intro-
ducing missing values if any index values were not already present:

In [ ]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

In [ ]:
obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0)

For ordered data like time series, it may be desirable to do some interpolation or filling
of values when reindexing. The **method** option allows us to do this, using a method such
as **ffill** which forward fills the values:

In [ ]:
obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

**Table 5-4 **lists available method options. At this time, interpolation more sophisticated
than forward- and backfilling would need to be applied after the fact.

**Table 5-4. reindex method (interpolation) options**

Argument | Description
--- | ---
ffill or pad | Fill (or carry) values forward
bfill or backfill | Fill (or carry) values backward

With DataFrame, **reindex** can alter either the (row) index, columns, or both. When
passed just a sequence, the rows are reindexed in the result:

In [ ]:
frame = DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'],
                  columns=['Ohio', 'Texas', 'California'])
frame

In [ ]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

The columns can be reindexed using the columns keyword:

In [ ]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

Both can be reindexed in one shot, though interpolation will only apply row-wise (axis
0):

In [ ]:
#frame.reindex(index=['a', 'b', 'c', 'd'], method='ffill', columns=states)
frame.reindex(index=['a', 'b', 'c', 'd'], columns=states).ffill()

As you’ll see soon, reindexing can be done more succinctly by label-indexing with `loc` :

In [ ]:
frame.loc[['a', 'b', 'c', 'd'], states]

**Table 5-5. reindex function arguments**

Argument | Description
--- | ---
index | New sequence to use as index. Can be Index instance or any other sequence-like Python data structure. An Index will be used exactly as is without any copying
method | Interpolation (fill) method, see Table 5-4 for options.
fill_value | Substitute value to use when introducing missing data by reindexing
limit | When forward- or backfilling, maximum size gap to fill
level | Match simple Index on level of MultiIndex, otherwise select subset of
copy | Do not copy underlying data if new index is equivalent to old index. True by default (i.e. always copy data).

### Dropping entries from an axis

Dropping one or more entries from an axis is easy if you have an index array or list
without those entries. As that can require a bit of munging and set logic, the drop
method will return a new object with the indicated value or values deleted from an axis:

In [ ]:
obj = Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c')
new_obj

In [ ]:
obj.drop(['d', 'c'])

With DataFrame, index values can be deleted from either axis:

In [ ]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                 columns=['one', 'two', 'three', 'four'])

In [ ]:
data.drop(['Colorado', 'Ohio'])

In [ ]:
data.drop('two', axis=1)

In [ ]:
data.drop(['two', 'four'], axis=1)

### Indexing, selection, and filtering

Series indexing ( `obj[...]` ) works analogously to NumPy array indexing, except you can
use the Series’s index values instead of only integers. Here are some examples this:

In [ ]:
obj = Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj['b']

In [ ]:
obj[1]

In [ ]:
obj[2:4]

In [ ]:
obj[['b', 'a', 'd']]

In [ ]:
obj[[1, 3]]

In [ ]:
obj[obj < 2]

Slicing with labels behaves differently than normal Python slicing in that the endpoint
is inclusive:

In [ ]:
obj['b':'c']

Setting using these methods works just as you would expect:

In [ ]:
obj['b':'c'] = 5
obj

As you’ve seen above, indexing into a DataFrame is for retrieving one or more columns
either with a single value or sequence:

In [ ]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data['two']

In [ ]:
data[['three', 'one']]

Indexing like this has a few special cases. First selecting rows by slicing or a boolean
array:

In [ ]:
data[:2]

In [ ]:
data[data['three'] > 5]

This might seem inconsistent to some readers, but this syntax arose out of practicality
and nothing more. Another use case is in indexing with a boolean DataFrame, such as
one produced by a scalar comparison:

In [ ]:
data < 5

In [ ]:
data[data < 5] = 0

In [ ]:
data

This is intended to make DataFrame syntactically more like an ndarray in this case.
For DataFrame label-indexing on the rows, I introduce the special indexing field `loc` . It
enables you to select a subset of the rows and columns from a DataFrame with NumPy-like notation plus axis labels. As I mentioned earlier, this is also a less verbose way to
do reindexing:

http://pandas.pydata.org/pandas-docs/stable/indexing.html#different-choices-for-indexing

In [ ]:
data.loc['Colorado', ['two', 'three']]

In [ ]:
data.ix[['Colorado', 'Utah'], [3, 0, 1]]

In [ ]:
data.iloc[2]

In [ ]:
data.loc[:'Utah', 'two']

In [ ]:
data.loc[data.three > 5, :'two']

**Table 5-6. Indexing options with DataFrame**

Type | Notes
--- | ---
obj[val]  |Select single column or sequence of columns from the DataFrame. Special case conveniences: boolean array (filter rows), slice (slice rows), or boolean DataFrame (set values based on some criterion).
obj.ix[val] | Selects single row of subset of rows from the DataFrame.
obj.ix[:, val] | Selects single column of subset of columns.
obj.ix[val1, val2] | Select both rows and columns. 
reindex method | Conform one or more axes to new indexes.
xs method | Select single row or column as a Series by label.
icol, irow methods | Select single column or row, respectively, as a Series by integer location.
get_value, set_value methods | Select single value by row and column label.

### Arithmetic and data alignment

One of the most important pandas features is the behavior of arithmetic between objects with different indexes. When adding together objects, if any index pairs are not
the same, the respective index in the result will be the union of the index pairs. Let’s
look at a simple example:

In [ ]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])

In [ ]:
s1

In [ ]:
s2

Adding these together yields:

In [ ]:
s1 + s2

The internal data alignment introduces NA values in the indices that don’t overlap.
Missing values propagate in arithmetic computations.
In the case of DataFrame, alignment is performed on both the rows and the columns:

In [ ]:
df1 = DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                index=['Ohio', 'Texas', 'Colorado'])
df2 = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1

In [ ]:
df2

Adding these together returns a DataFrame whose index and columns are the unions
of the ones in each DataFrame:

In [ ]:
df1 + df2

#### Arithmetic methods with fill values

In arithmetic operations between differently-indexed objects, you might want to fill
with a special value, like 0, when an axis label is found in one object but not the other:

In [ ]:
df1 = DataFrame(np.arange(12.).reshape((3, 4)), columns=list('abcd'))
df2 = DataFrame(np.arange(20.).reshape((4, 5)), columns=list('abcde'))
df1

In [ ]:
df2

Adding these together results in NA values in the locations that don’t overlap:

In [ ]:
df1 + df2

Using the add method on df1 , I pass df2 and an argument to fill_value :

In [ ]:
df1.add(df2, fill_value=0)

Relatedly, when reindexing a Series or DataFrame, you can also specify a different fill
value:

In [ ]:
df1.reindex(columns=df2.columns, fill_value=0)

**Table 5-7. Flexible arithmetic methods**

Method | Description
--- | ---
add | Method for addition (+)
sub | Method for subtraction (-)
div | Method for division (/)
mul | Method for multiplication (*)

#### Operations between DataFrame and Series

As with NumPy arrays, arithmetic between DataFrame and Series is well-defined. First,
as a motivating example, consider the difference between a 2D array and one of its rows:

In [ ]:
arr = np.arange(12.).reshape((3, 4))
arr

In [ ]:
arr[0]

In [ ]:
arr - arr[0]

This is referred to as broadcasting and is explained in more detail in Chapter 12. Op-
erations between a DataFrame and a Series are similar:

In [ ]:
frame = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0]
frame

In [ ]:
series

By default, arithmetic between DataFrame and Series matches the index of the Series
on the DataFrame's columns, broadcasting down the rows:

In [ ]:
frame - series

If an index value is not found in either the DataFrame’s columns or the Series’s index,
the objects will be reindexed to form the union:

In [ ]:
series2 = Series(range(3), index=['b', 'e', 'f'])
frame + series2

If you want to instead broadcast over the columns, matching on the rows, you have to
use one of the arithmetic methods. For example:

In [ ]:
series3 = frame['d']
frame

In [ ]:
series3

In [ ]:
frame.sub(series3, axis=0)

The axis number that you pass is the axis to match on. In this case we mean to match
on the DataFrame’s row index and broadcast across.

### Function application and mapping

NumPy ufuncs (element-wise array methods) work fine with pandas objects:

In [ ]:
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [ ]:
frame

In [ ]:
np.abs(frame)

Another frequent operation is applying a function on 1D arrays to each column or row.
DataFrame’s **apply** method does exactly this:

In [ ]:
f = lambda x: x.max() - x.min()

In [ ]:
frame.apply(f)

In [ ]:
frame.apply(f, axis=1)

Many of the most common array statistics (like **sum** and **mean** ) are DataFrame methods,
so using **apply** is not necessary.
The function passed to **apply** need not return a scalar value, it can also return a Series
with multiple values:

In [ ]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

Element-wise Python functions can be used, too. Suppose you wanted to compute a
formatted string from each floating point value in **frame** . You can do this with **applymap** :

In [ ]:
format = lambda x: '%.2f' % x
frame.applymap(format)

In [ ]:
frame['e'].map(format)

### Sorting and ranking

Sorting a data set by some criterion is another important built-in operation. To sort
lexicographically by row or column index, use the sort_index method, which returns
a new, sorted object:

In [ ]:
obj = Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

With a DataFrame, you can sort by index on either axis:

In [ ]:
frame = DataFrame(np.arange(8).reshape((2, 4)), index=['three', 'one'],
                  columns=['d', 'a', 'b', 'c'])
frame.sort_index()

In [ ]:
frame.sort_index(axis=1)

The data is sorted in ascending order by default, but can be sorted in descending order,
too:

In [ ]:
frame.sort_index(axis=1, ascending=False)

To sort a Series by its values, use its order method:

In [ ]:
obj = Series([4, 7, -3, 2])
obj.order()

Any missing values are sorted to the end of the Series by default:

In [ ]:
obj = Series([4, np.nan, 7, np.nan, -3, 2])
obj.order()

On DataFrame, you may want to sort by the values in one or more columns. To do so,
pass one or more column names to the by option:

In [ ]:
frame = DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

In [ ]:
frame.sort_index(by='b')

To sort by multiple columns, pass a list of names:

In [ ]:
frame.sort_index(by=['a', 'b'])

*Ranking* is closely related to sorting, assigning ranks from one through the number of
valid data points in an array. It is similar to the indirect sort indices produced by
**numpy.argsort** , except that ties are broken according to a rule. The **rank** methods for
Series and DataFrame are the place to look; by default **rank** breaks ties by assigning
each group the mean rank:

In [ ]:
obj = Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

Ranks can also be assigned according to the order they’re observed in the data:

In [ ]:
obj.rank(method='first')

Naturally, you can rank in descending order, too:

In [ ]:
obj.rank(ascending=False, method='max')

See **Table 5-8** for a list of tie-breaking methods available. DataFrame can compute ranks
over the rows or the columns:

In [ ]:
frame = DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                   'c': [-2, 5, 8, -2.5]})
frame

In [ ]:
frame.rank(axis=1)

**Table 5-8. Tie-breaking methods with rank**

Method | Description
--- | ---
'average' | Default: assign the average rank to each entry in the equal group.
'min' | Use the minimum rank for the whole group.
'max' | Use the maximum rank for the whole group.
'first' | Assign ranks in the order the values appear in the data.

### Axis indexes with duplicate values

Up until now all of the examples I’ve showed you have had unique axis labels (index
values). While many pandas functions (like reindex ) require that the labels be unique,
it’s not mandatory. Let’s consider a small Series with duplicate indices:

In [ ]:
obj = Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

The index’s is_unique property can tell you whether its values are unique or not:

In [ ]:
obj.index.is_unique

Data selection is one of the main things that behaves differently with duplicates. Indexing a value with multiple entries returns a Series while single entries return a scalar
value:

In [ ]:
obj['a']

In [ ]:
obj['c']

The same logic extends to indexing rows in a DataFrame:

In [ ]:
df = DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

In [ ]:
df.loc['b']

## Summarizing and computing descriptive statistics

pandas objects are equipped with a set of common mathematical and statistical methods. Most of these fall into the category of *reductions* or *summary statistics*, methods
that extract a single value (like the sum or mean) from a Series or a Series of values from
the rows or columns of a DataFrame. Compared with the equivalent methods of vanilla
NumPy arrays, they are all built from the ground up to exclude missing data. Consider
a small DataFrame:

In [ ]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5],
                [np.nan, np.nan], [0.75, -1.3]],
               index=['a', 'b', 'c', 'd'],
               columns=['one', 'two'])
df

Calling DataFrame’s sum method returns a Series containing column sums:

In [ ]:
df.sum()

Passing axis=1 sums over the rows instead:

In [ ]:
df.sum(axis=1)

NA values are excluded unless the entire slice (row or column in this case) is NA. This
can be disabled using the skipna option:

In [ ]:
df.mean(axis=1, skipna=False)

See **Table 5-9** for a list of common options for each reduction method options.

**Table 5-9. Options for reduction methods**

Method | Description
--- | ---
axis | Axis to reduce over. 0 for DataFrame’s rows and 1 for columns.
skipna | Exclude missing values, True by default.
level | Reduce grouped by level if the axis is hierarchically-indexed (MultiIndex).

Some methods, like idxmin and idxmax , return indirect statistics like the index value
where the minimum or maximum values are attained:

In [ ]:
df.idxmax()

In [ ]:
df.apply(np.argmax)

In [ ]:
df.cumsum()

Another type of method is neither a reduction nor an accumulation. describe is one
such example, producing multiple summary statistics in one shot:

In [ ]:
df.describe()

On non-numeric data, describe produces alternate summary statistics:

In [ ]:
obj = Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

See **Table 5-10** for a full list of summary statistics and related methods.

**Table 5-10. Descriptive and summary statistics**

Method | Description
--- | ---
count | Number of non-NA values
describe | Compute set of summary statistics for Series or each DataFrame column
min, max | Compute minimum and maximum values
argmin, argmax | Compute index locations (integers) at which minimum or maximum value obtained, respectively
idxmin, idxmax | Compute index values at which minimum or maximum value obtained, respectively
quantile | Compute sample quantile ranging from 0 to 1
sum | Sum of values
mean | Mean of values
median | Arithmetic median (50% quantile) of values
mad | Mean absolute deviation from mean value
var | Sample variance of values
std | Sample standard deviation of values
skew | Sample skewness (3rd moment) of values
kurt | Sample kurtosis (4th moment) of values
cumsum | Cumulative sum of values
cummin, cummax | Cumulative minimum or maximum of values, respectively
cumprod | Cumulative product of values
diff | Compute 1st arithmetic difference (useful for time series)
pct_change | Compute percent changes

### Correlation and covariance

Some summary statistics, like correlation and covariance, are computed from pairs of
arguments. Let’s consider some DataFrames of stock prices and volumes obtained from
Yahoo! Finance:

In [ ]:
!pip install pandas-datareader

In [ ]:
import pandas_datareader as pdr

all_data = {}
for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']:
    all_data[ticker] = pdr.get_data_yahoo(ticker)

price = DataFrame({tic: data['Adj Close']
                   for tic, data in all_data.iteritems()})
volume = DataFrame({tic: data['Volume']
                    for tic, data in all_data.iteritems()})

In [ ]:
returns = price.pct_change()
returns.tail()

In [ ]:
returns.MSFT.corr(returns.IBM)

In [ ]:
returns.MSFT.cov(returns.IBM)

In [ ]:
returns.corr()

In [ ]:
returns.cov()

In [ ]:
returns.corrwith(returns.IBM)

In [ ]:
returns.corrwith(volume)

### Unique values, value counts, and membership

In [ ]:
obj = Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

In [ ]:
uniques = obj.unique()
uniques

In [ ]:
obj.value_counts()

In [ ]:
pd.value_counts(obj.values, sort=False)

In [ ]:
mask = obj.isin(['b', 'c'])
mask

In [ ]:
obj[mask]

In [ ]:
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
                  'Qu2': [2, 3, 1, 2, 3],
                  'Qu3': [1, 5, 2, 4, 4]})
data

In [ ]:
result = data.apply(pd.value_counts).fillna(0)
result

## Handling missing data

In [ ]:
string_data = Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

In [ ]:
string_data.isnull()

In [ ]:
string_data[0] = None
string_data.isnull()

### Filtering out missing data

In [ ]:
from numpy import nan as NA
data = Series([1, NA, 3.5, NA, 7])
data.dropna()

In [ ]:
data[data.notnull()]

In [ ]:
data = DataFrame([[1., 6.5, 3.], [1., NA, NA],
                  [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

In [ ]:
cleaned

In [ ]:
data.dropna(how='all')

In [ ]:
data[4] = NA
data

In [ ]:
data.dropna(axis=1, how='all')

In [ ]:
df = DataFrame(np.random.randn(7, 3))
df.ix[:4, 1] = NA; df.ix[:2, 2] = NA
df

In [ ]:
df.dropna(thresh=3)

### Filling in missing data

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 3: -1})

In [ ]:
# always returns a reference to the filled object
_ = df.fillna(0, inplace=True)
df

In [ ]:
df = DataFrame(np.random.randn(6, 3))
df.ix[2:, 1] = NA; df.ix[4:, 2] = NA
df

In [ ]:
df.fillna(method='ffill')

In [ ]:
df.fillna(method='ffill', limit=2)

In [ ]:
data = Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

## Hierarchical indexing

In [ ]:
data = Series(np.random.randn(10),
              index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                     [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

In [ ]:
data.index

In [ ]:
data['b']

In [ ]:
data['b':'c']

In [ ]:
data.ix[['b', 'd']]

In [ ]:
data[:, 2]

In [ ]:
data.unstack()

In [ ]:
data.unstack().stack()

In [ ]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']])
frame

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

In [ ]:
frame['Ohio']

### Reordering and sorting levels

In [ ]:
frame.swaplevel('key1', 'key2')

In [ ]:
frame.sortlevel(1)

In [ ]:
frame.swaplevel(0, 1).sortlevel(0)

### Summary statistics by level

In [ ]:
frame.sum(level='key2')

In [ ]:
frame.sum(level='color', axis=1)

### Using a DataFrame's columns

In [ ]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                   'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd': [0, 1, 2, 0, 1, 2, 3]})
frame

In [ ]:
frame2 = frame.set_index(['c', 'd'])
frame2

In [ ]:
frame.set_index(['c', 'd'], drop=False)

In [ ]:
frame2.reset_index()

## Other pandas topics

### Integer indexing

In [ ]:
ser = Series(np.arange(3.))
ser.iloc[-1]

In [ ]:
ser

In [ ]:
ser2 = Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

In [ ]:
ser.ix[:1]

In [ ]:
ser3 = Series(range(3), index=[-5, 1, 3])
ser3.iloc[2]

In [ ]:
frame = DataFrame(np.arange(6).reshape((3, 2)), index=[2, 0, 1])
frame.iloc[0]

### Panel data

In [ ]:
import pandas.io.data as web

pdata = pd.Panel(dict((stk, web.get_data_yahoo(stk))
                       for stk in ['AAPL', 'GOOG', 'MSFT', 'DELL']))

In [ ]:
pdata

In [ ]:
pdata = pdata.swapaxes('items', 'minor')
pdata['Adj Close']

In [ ]:
pdata.ix[:, '6/1/2012', :]

In [ ]:
pdata.ix['Adj Close', '5/22/2012':, :]

In [ ]:
stacked = pdata.ix[:, '5/30/2012':, :].to_frame()
stacked

In [ ]:
stacked.to_panel()